# 项目主要爬取全国367个城市2019年春节期间的空气质量指数数据，并分析烟花爆竹对空气质量的影响，以及城市禁放烟花爆竹是否有效
## 主要的文件为：

- Fireworks and Pm2.5 v2.0.ipynb：Jupyter notebook代码，对空气质量指数数据进行分析
- AQI_Data.csv: 2019年除夕到初七的6.8万条AQI数据

#### 运行环境：

- python3.7

#### 需要安装的包：

- pyecharts(v1.x或以上)
- pandas
- numpy

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyecharts.options as opts

In [43]:
data = pd.read_csv('AQI_Data.csv')

In [44]:
data.head()

,city,AQI,PM2.5,PM10,SO2,NO2,CO,O3,main_pollution,time
0,北京市,61,14,71,3,12,0.3,61,PM10,2019-02-04 00:00:00
1,天津市,104,41,157,8,21,0.9,53,PM10,2019-02-04 00:00:00
2,石家庄市,125,36,200,12,47,0.9,30,PM10,2019-02-04 00:00:00
3,唐山市,66,42,81,22,16,1.0,52,PM10,2019-02-04 00:00:00
4,秦皇岛市,55,25,60,12,22,0.9,48,PM10,2019-02-04 00:00:00


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71825 entries, 0 to 71824
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   city            71825 non-null  object
 1   AQI             71825 non-null  object
 2   PM2.5           71825 non-null  object
 3   PM10            71825 non-null  object
 4   SO2             71825 non-null  object
 5   NO2             71825 non-null  object
 6   CO              71825 non-null  object
 7   O3              71825 non-null  object
 8   main_pollution  71825 non-null  object
 9   time            71825 non-null  object
dtypes: object(10)
memory usage: 5.5+ MB


In [46]:
data['time'].unique()

array(['2019-02-04 00:00:00', '2019-02-04 01:00:00',
       '2019-02-04 02:00:00', '2019-02-04 03:00:00',
       '2019-02-04 04:00:00', '2019-02-04 05:00:00',
       '2019-02-04 06:00:00', '2019-02-04 07:00:00',
       '2019-02-04 08:00:00', '2019-02-04 09:00:00',
       '2019-02-04 10:00:00', '2019-02-04 11:00:00',
       '2019-02-04 12:00:00', '2019-02-04 13:00:00',
       '2019-02-04 14:00:00', '2019-02-04 15:00:00',
       '2019-02-04 16:00:00', '2019-02-04 17:00:00',
       '2019-02-04 18:00:00', '2019-02-04 19:00:00',
       '2019-02-04 20:00:00', '2019-02-04 21:00:00',
       '2019-02-04 22:00:00', '2019-02-04 23:00:00',
       '2019-02-05 00:00:00', '2019-02-05 01:00:00',
       '2019-02-05 02:00:00', '2019-02-05 03:00:00',
       '2019-02-05 04:00:00', '2019-02-05 05:00:00',
       '2019-02-05 06:00:00', '2019-02-05 07:00:00',
       '2019-02-05 08:00:00', '2019-02-05 09:00:00',
       '2019-02-05 10:00:00', '2019-02-05 11:00:00',
       '2019-02-05 12:00:00', '2019-02-05 13:0

In [47]:
# 数据清洗
data['time'] = pd.to_datetime(data['time'])
data = data[data['time']<=pd.to_datetime('2019-02-11 23:59:59')]  # 选取2月4日——2月12日的数据
#data.set_index(data["time"], inplace=True)
data = data.replace('—', np.nan)
for col in ['AQI', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']:
    data[col] = data[col].astype(float)
data

,city,AQI,PM2.5,PM10,SO2,NO2,CO,O3,main_pollution,time
0,北京市,61.0,14.0,71.0,3.0,12.0,0.3,61.0,PM10,2019-02-04 00:00:00
1,天津市,104.0,41.0,157.0,8.0,21.0,0.9,53.0,PM10,2019-02-04 00:00:00
2,石家庄市,125.0,36.0,200.0,12.0,47.0,0.9,30.0,PM10,2019-02-04 00:00:00
3,唐山市,66.0,42.0,81.0,22.0,16.0,1.0,52.0,PM10,2019-02-04 00:00:00
4,秦皇岛市,55.0,25.0,60.0,12.0,22.0,0.9,48.0,PM10,2019-02-04 00:00:00
...,...,...,...,...,...,...,...,...,...,...
68150,伊犁哈萨克州,85.0,63.0,78.0,37.0,32.0,1.2,92.0,PM2.5,2019-02-11 23:00:00
68151,塔城地区,20.0,13.0,20.0,3.0,12.0,1.3,59.0,NaN,2019-02-11 23:00:00
68152,阿勒泰地区,29.0,20.0,28.0,1.0,14.0,1.3,44.0,NaN,2019-02-11 23:00:00
68153,石河子市,224.0,174.0,227.0,12.0,52.0,1.6,43.0,PM2.5,2019-02-11 23:00:00


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68155 entries, 0 to 68154
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   city            68155 non-null  object        
 1   AQI             67903 non-null  float64       
 2   PM2.5           67815 non-null  float64       
 3   PM10            66437 non-null  float64       
 4   SO2             67832 non-null  float64       
 5   NO2             67852 non-null  float64       
 6   CO              67838 non-null  float64       
 7   O3              67842 non-null  float64       
 8   main_pollution  41176 non-null  object        
 9   time            68155 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 5.7+ MB


In [49]:
data['city'].unique()

array(['北京市', '天津市', '石家庄市', '唐山市', '秦皇岛市', '邯郸市', '邢台市', '保定市', '承德市',
       '沧州市', '廊坊市', '衡水市', '张家口市', '太原市', '大同市', '阳泉市', '长治市', '晋城市',
       '朔州市', '晋中市', '运城市', '忻州市', '临汾市', '吕梁市', '呼和浩特市', '包头市', '乌海市',
       '赤峰市', '通辽市', '鄂尔多斯市', '呼伦贝尔市', '巴彦淖尔市', '乌兰察布市', '兴安盟', '锡林郭勒盟',
       '阿拉善盟', '沈阳市', '大连市', '瓦房店市', '鞍山市', '抚顺市', '本溪市', '丹东市', '锦州市',
       '营口市', '阜新市', '辽阳市', '盘锦市', '铁岭市', '朝阳市', '葫芦岛市', '长春市', '吉林市',
       '四平市', '辽源市', '通化市', '白山市', '松原市', '白城市', '延边州', '哈尔滨市', '齐齐哈尔市',
       '鸡西市', '鹤岗市', '双鸭山市', '大庆市', '伊春市', '佳木斯市', '七台河市', '牡丹江市', '黑河市',
       '绥化市', '大兴安岭地区', '上海市', '南京市', '无锡市', '江阴市', '宜兴市', '徐州市', '常州市',
       '溧阳市', '金坛市', '苏州市', '常熟市', '张家港市', '昆山市', '吴江市', '太仓市', '南通市',
       '海门市', '连云港市', '淮安市', '盐城市', '扬州市', '镇江市', '句容市', '泰州市', '宿迁市',
       '杭州市', '富阳市', '临安市', '宁波市', '温州市', '嘉兴市', '湖州市', '诸暨市', '金华市',
       '义乌市', '衢州市', '舟山市', '台州市', '丽水市', '绍兴市', '合肥市', '芜湖市', '蚌埠市',
       '淮南市', '马鞍山市', '淮北市', '铜陵市', '安庆市', '黄山市', '滁州市', '阜阳市', '宿州

In [50]:
data.to_csv('data_clean.csv', index=False)

#### 一、数据详情
- 共有68155条数据，时间为2月4日到2月11日，每隔1小时收集一次，囊括全国367个城市
- 包含字段：城市、空气质量指数、PM2.5指数、PM10指数、SO2指数、NO2指数、CO指数、O3指数、主要污染源、时间

#### 二、问题
1. 燃放烟花爆竹真的对空气质量有影响吗？
2. 烟花爆竹对空气质量的影响体现在哪些指标上？
3. 烟花爆竹对空气质量的污染程度有多大？
4. 哪些区域污染最严重(轻)？
5. 哪些城市属于一秒破功型？
6. 除夕中午到初一中午的超标城市个数



##### 1. 燃放烟花爆竹真的对空气质量有影响吗？

In [51]:
AQI_total_mean = data[data['time']<=pd.to_datetime('2019-02-06 23:59:59')].groupby(['time'])['AQI'].mean()

In [52]:
AQI_total_mean.index

DatetimeIndex(['2019-02-04 00:00:00', '2019-02-04 01:00:00',
               '2019-02-04 02:00:00', '2019-02-04 03:00:00',
               '2019-02-04 04:00:00', '2019-02-04 05:00:00',
               '2019-02-04 06:00:00', '2019-02-04 07:00:00',
               '2019-02-04 08:00:00', '2019-02-04 09:00:00',
               '2019-02-04 10:00:00', '2019-02-04 11:00:00',
               '2019-02-04 12:00:00', '2019-02-04 13:00:00',
               '2019-02-04 14:00:00', '2019-02-04 15:00:00',
               '2019-02-04 16:00:00', '2019-02-04 17:00:00',
               '2019-02-04 18:00:00', '2019-02-04 19:00:00',
               '2019-02-04 20:00:00', '2019-02-04 21:00:00',
               '2019-02-04 22:00:00', '2019-02-04 23:00:00',
               '2019-02-05 00:00:00', '2019-02-05 01:00:00',
               '2019-02-05 02:00:00', '2019-02-05 03:00:00',
               '2019-02-05 04:00:00', '2019-02-05 05:00:00',
               '2019-02-05 06:00:00', '2019-02-05 07:00:00',
               '2019-02-

In [53]:
from datetime import datetime
time_str = AQI_total_mean.index.strftime('%Y-%m-%d %H:%M:%S')
print(time_str)

Index(['2019-02-04 00:00:00', '2019-02-04 01:00:00', '2019-02-04 02:00:00',
       '2019-02-04 03:00:00', '2019-02-04 04:00:00', '2019-02-04 05:00:00',
       '2019-02-04 06:00:00', '2019-02-04 07:00:00', '2019-02-04 08:00:00',
       '2019-02-04 09:00:00', '2019-02-04 10:00:00', '2019-02-04 11:00:00',
       '2019-02-04 12:00:00', '2019-02-04 13:00:00', '2019-02-04 14:00:00',
       '2019-02-04 15:00:00', '2019-02-04 16:00:00', '2019-02-04 17:00:00',
       '2019-02-04 18:00:00', '2019-02-04 19:00:00', '2019-02-04 20:00:00',
       '2019-02-04 21:00:00', '2019-02-04 22:00:00', '2019-02-04 23:00:00',
       '2019-02-05 00:00:00', '2019-02-05 01:00:00', '2019-02-05 02:00:00',
       '2019-02-05 03:00:00', '2019-02-05 04:00:00', '2019-02-05 05:00:00',
       '2019-02-05 06:00:00', '2019-02-05 07:00:00', '2019-02-05 08:00:00',
       '2019-02-05 09:00:00', '2019-02-05 10:00:00', '2019-02-05 11:00:00',
       '2019-02-05 12:00:00', '2019-02-05 13:00:00', '2019-02-05 14:00:00',
       '2019

In [55]:
x = ['2019-02-04 00:00:00', '2019-02-04 01:00:00', '2019-02-04 02:00:00',
       '2019-02-04 03:00:00', '2019-02-04 04:00:00', '2019-02-04 05:00:00',
       '2019-02-04 06:00:00', '2019-02-04 07:00:00', '2019-02-04 08:00:00',
       '2019-02-04 09:00:00', '2019-02-04 10:00:00', '2019-02-04 11:00:00',
       '2019-02-04 12:00:00', '2019-02-04 13:00:00', '2019-02-04 14:00:00',
       '2019-02-04 15:00:00', '2019-02-04 16:00:00', '2019-02-04 17:00:00',
       '2019-02-04 18:00:00', '2019-02-04 19:00:00', '2019-02-04 20:00:00',
       '2019-02-04 21:00:00', '2019-02-04 22:00:00', '2019-02-04 23:00:00',
       '2019-02-05 00:00:00', '2019-02-05 01:00:00', '2019-02-05 02:00:00',
       '2019-02-05 03:00:00', '2019-02-05 04:00:00', '2019-02-05 05:00:00',
       '2019-02-05 06:00:00', '2019-02-05 07:00:00', '2019-02-05 08:00:00',
       '2019-02-05 09:00:00', '2019-02-05 10:00:00', '2019-02-05 11:00:00',
       '2019-02-05 12:00:00', '2019-02-05 13:00:00', '2019-02-05 14:00:00',
       '2019-02-05 15:00:00', '2019-02-05 16:00:00', '2019-02-05 17:00:00',
       '2019-02-05 18:00:00', '2019-02-05 19:00:00', '2019-02-05 20:00:00',
       '2019-02-05 21:00:00', '2019-02-05 22:00:00', '2019-02-05 23:00:00',
       '2019-02-06 00:00:00', '2019-02-06 01:00:00', '2019-02-06 02:00:00',
       '2019-02-06 03:00:00', '2019-02-06 04:00:00', '2019-02-06 05:00:00',
       '2019-02-06 06:00:00', '2019-02-06 07:00:00', '2019-02-06 08:00:00',
       '2019-02-06 09:00:00', '2019-02-06 10:00:00', '2019-02-06 11:00:00',
       '2019-02-06 12:00:00', '2019-02-06 13:00:00', '2019-02-06 14:00:00',
       '2019-02-06 15:00:00', '2019-02-06 16:00:00', '2019-02-06 17:00:00',
       '2019-02-06 18:00:00', '2019-02-06 19:00:00', '2019-02-06 20:00:00',
       '2019-02-06 21:00:00', '2019-02-06 22:00:00', '2019-02-06 23:00:00']

In [57]:
from pyecharts.charts import Line
line =(
    Line()
    .add_xaxis(x)
    .add_yaxis('',np.round(AQI_total_mean.values,0),is_smooth=True,
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max')]))# 设置线为平滑的
    .set_global_opts(
        title_opts=opts.TitleOpts(title='全国春节期间空气质量指数总体趋势',subtitle="2019年除夕到初二"),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
line.render('全国春节期间空气质量指数总体趋势.html')

'C:\\Users\\阿泽\\Desktop\\全国春节期间空气质量指数总体趋势.html'

##### 2. 烟花爆竹对空气质量的影响体现在哪些指标上？

In [59]:
data_total_idx = data[data['time']<=pd.to_datetime('2019-02-06 23:59:59')].groupby(['time']).mean()

In [60]:
data_total_idx.columns

Index(['AQI', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3'], dtype='object')

In [68]:
(
    Line()
    .add_xaxis(x)
    .add_yaxis(
        series_name='PM2.5',
        stack='AQI',
        y_axis=data_total_idx['PM2.5'].values,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name='PM10',
        stack='AQI',
        y_axis=data_total_idx['PM10'].values,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name='SO2',
        stack='AQI',
        y_axis=data_total_idx['SO2'].values,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name='NO2',
        stack='AQI',
        y_axis=data_total_idx['NO2'].values,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name='CO',
        stack='AQI',
        y_axis=data_total_idx['CO'].values,
        label_opts=opts.LabelOpts(is_show=False),
    )
     .add_yaxis(
        series_name='O3',
        stack='AQI',
        y_axis=data_total_idx['O3'].values,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="全国春节期间空气质量各指标（2019年除夕到初二）",subtitle="单位:μg/m3(CO为mg/m3)"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        legend_opts=opts.LegendOpts(pos_right=50)
    )
    .render("全国春节期间空气质量各指标（2019年除夕到初二）.html")
)

'C:\\Users\\阿泽\\Desktop\\全国春节期间空气质量各指标（2019年除夕到初二）.html'

In [56]:
main_pol = data.loc[data['time']==pd.to_datetime('2019-02-05 02:00:00'), 'main_pollution'].value_counts()

In [57]:
main_pol

PM2.5         243
PM10           97
PM10,PM2.5     10
Name: main_pollution, dtype: int64

In [67]:
main_pol.values
bar_y = [243,  97,  10]

In [68]:
bar_x = ['PM2.5', 'PM10', 'PM10,PM2.5']
bar_x

['PM2.5', 'PM10', 'PM10,PM2.5']

In [75]:
from pyecharts.charts import Bar

bar1 = (
    Bar()
    .add_xaxis(bar_x)
    .add_yaxis("", bar_y,color=['#130f40'])
    .set_global_opts(title_opts=opts.TitleOpts(title="全国城市首要污染物", subtitle="2019年2月5日凌晨2点"))
    .render("全国城市首要污染物.html")
)

##### 3. 烟花爆竹对空气质量的污染程度有多大？

In [78]:
data2 = data[data['time']<=pd.to_datetime('2019-02-05 23:59:59')]
data_AQI_min = data2.groupby('city')['AQI'].min()
data_AQI_max = data2.groupby('city')['AQI'].max()

In [79]:
data_AQI_times = np.round(data_AQI_max/data_AQI_min, 1)

In [80]:
data_AQI_times_counts = data_AQI_times.value_counts(bins=[1,5,10,15,20,25])
data_AQI_times_counts

(0.999, 5.0]    238
(5.0, 10.0]      88
(10.0, 15.0]     28
(15.0, 20.0]      9
(20.0, 25.0]      4
Name: AQI, dtype: int64

In [88]:
data_AQI_times_counts.values
bar_y = [238,  88,  28,   9,   4]

In [91]:
times = ['1-5倍','5-10倍','10-15倍','15-20倍','20-25倍']
bar2 = (
    Bar()
    .add_xaxis(times)
    .add_yaxis("", bar_y,color = ['#130f40'])
    .set_global_opts(title_opts=opts.TitleOpts(title="全国除夕和春节期间空气质量最高最低比的城市数量", subtitle="2019年除夕至初一"))
    .render("全国除夕和春节期间空气质量最高最低比的城市数量.html")
)

##### 4. 哪些区域污染最严重（轻）？

In [10]:
start_time = pd.to_datetime('2019-02-04 20:00:00')
end_time = pd.to_datetime('2019-02-05 04:00:00')
AQI_by_city = data[(data['time']>=start_time)&(data['time']<=end_time)].groupby('city')['AQI'].mean()

In [12]:
attr, value = AQI_by_city.index, AQI_by_city.values

In [37]:
from pyecharts.charts import Geo


geo = (
    Geo(init_opts=opts.InitOpts(bg_color="#404a59"))
    .add_schema(maptype="china",itemstyle_opts=opts.ItemStyleOpts(color="#323c48", border_color="#111"))
    .add("", [list(z) for z in zip(attr,value)])
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(textstyle_opts=opts.TextStyleOpts(color="#fff"),is_piecewise=True,min_=0,max_=500,split_number=10),
        title_opts=opts.TitleOpts(title_textstyle_opts=opts.TextStyleOpts(color="#fff"),title="全国各城市空气质量指数",subtitle="除夕晚上20点至初一凌晨4点平均值",pos_left="center"),
    )
    .render("全国各城市空气质量指数.html")
)

In [75]:
AQI_by_city.nlargest(10)

city
阿克苏地区    454.333333
库尔勒市     436.111111
呼和浩特市    394.333333
和田地区     357.666667
安康市      343.888889
白山市      340.333333
亳州市      320.333333
喀什地区     320.111111
包头市      295.666667
咸阳市      288.222222
Name: AQI, dtype: float64

##### 5. 哪些城市属于一秒破功型？

In [138]:
city1 = data.loc[(data['time'] == pd.to_datetime('2019-02-05 02:00:00'))&(data['AQI']<100), 'city']

In [162]:
data3 = data[(data['time'] >= pd.to_datetime('2019-02-05 18:00:00'))&
            (data['time'] <= pd.to_datetime('2019-02-06 23:59:59'))]

In [166]:
city2 = data3.loc[data3['AQI']>200, 'city']

In [167]:
list(set(city1).intersection(set(city2)))

['那曲地区', '铜川市', '郑州市', '白城市', '中卫市', '长春市', '吴忠市', '忻州市', '晋城市']

In [171]:
city_aqi_1 = data[(data['city']=='郑州市')&(data['time']<=pd.to_datetime('2019-02-06 23:59:59'))]
city_aqi_2 = data[(data['city']=='长春市')&(data['time']<=pd.to_datetime('2019-02-06 23:59:59'))]
city_aqi_3 = data[(data['city']=='忻州市')&(data['time']<=pd.to_datetime('2019-02-06 23:59:59'))]
city_aqi_4 = data[(data['city']=='晋城市')&(data['time']<=pd.to_datetime('2019-02-06 23:59:59'))]

In [172]:
(
    Line()
    .add_xaxis(x)
    .add_yaxis(
        series_name='郑州市',
        stack='AQI',
        y_axis=city_aqi_1['AQI'].values,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name='长春市',
        stack='AQI',
        y_axis=city_aqi_2['AQI'].values,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name='忻州市',
        stack='AQI',
        y_axis=np.round(city_aqi_3['AQI'].values,0),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name='晋城市',
        stack='AQI',
        y_axis=np.round(city_aqi_4['AQI'].values,0),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="春节期间空气质量指数",subtitle="2019年除夕到初二"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
    )
    .render("春节期间空气质量指数一秒破功型.html")
)

'C:\\Users\\阿泽\\Desktop\\春节期间空气质量指数一秒破功型.html'

##### 6. 除夕中午到初一中午的超标城市个数

In [5]:
start_time = pd.to_datetime('2019-02-04 12:00:00')
end_time = pd.to_datetime('2019-02-05 12:00:00')

In [12]:
city_over_new_year = data.loc[(data['time']>=start_time)&(data['time']<=end_time)&(data['AQI']>100)].groupby('time')['AQI'].count()

In [23]:
city_over_new_year.values

array([125, 118, 115, 110, 108, 103, 110, 121, 143, 178, 197, 205, 226,
       255, 255, 234, 219, 207, 199, 195, 204, 210, 205, 202, 192],
      dtype=int64)

In [24]:
bar3_y = [125, 118, 115, 110, 108, 103, 110, 121, 143, 178, 197, 205, 226,
       255, 255, 234, 219, 207, 199, 195, 204, 210, 205, 202, 192]

In [22]:
from pyecharts.charts import Bar, Line

new_year_time = ['除夕12点','除夕13点','除夕14点','除夕15点','除夕16点','除夕17点','除夕18点','除夕19点',
        '除夕20点','除夕21点','除夕22点','除夕23点','春节0点','春节1点','春节2点','春节3点',
        '春节4点','春节5点','春节6点','春节7点','春节8点','春节9点','春节10点','春节11点','春节12点']
bar3 = (
    Bar()
    .add_xaxis(new_year_time)
    .add_yaxis("", bar3_y)
    .extend_axis(
        yaxis=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} °C"), interval=5
        )
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="全国除夕中午至初一中午空气质量超标城市数量"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} ml")),
    )
)

line = Line().add_xaxis(new_year_time).add_yaxis("", bar3_y)
bar3.overlap(line)
bar3.render("全国除夕中午至初一中午空气质量超标城市数量.html")

'C:\\Users\\阿泽\\Desktop\\全国除夕中午至初一中午空气质量超标城市数量.html'